In [16]:
# Import packages & set workspace
import os
import requests as r
from zipfile import ZipFile

import arcpy
from arcpy.sa import *

arcpy.env.workspace = r'C:\Users\mmMary\Documents\8990\Map\Weather_Map.aprx'

# Step 1. Data collection & preparation

Climate normals files must be downloaded, unzipped, & projected. They are sourced from: https://prism.oregonstate.edu/normals 


Temporal scales are Monthly & Annual 
#### Variables are: 
    * Precipitation
    * Mean Temperature
    * Minimum Temperature
    * Maximum Temperature

### Generate list for pulling all monthly & annual variables for normals
  Number 14 denotes the annual normals whereas the months are 01 - 12

In [21]:
# Generate list for pulling all monthly & annual variables for normals

date_list = ["%.2d" % i for i in range(13)]
date_list.pop(0)
# Add for the 14 annual normals to the list
date_list.append('14')

print(date_list)

# https://stackoverflow.com/questions/12030074/generate-list-of-numbers-in-specific-format

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '14']


### Make an API call to download all data from the PRISM website

In [93]:
# Sample url: 'http://services.nacse.org/prism/data/public/4km/<element>/<date>' 

# Set a list of climate variables, base URL
element = ["ppt_", "tmean_", "tmax_", "tmin_"]
base = "http://services.nacse.org/prism/data/public/normals/4km"

for e in element:
    for i in date_list:
        url = os.path.join(base, e + i)
        r.get(base + element + date)   # API call for the data 
        print(url) 
    

http://services.nacse.org/prism/data/public/normals/4km\ppt_01
http://services.nacse.org/prism/data/public/normals/4km\ppt_02
http://services.nacse.org/prism/data/public/normals/4km\ppt_03
http://services.nacse.org/prism/data/public/normals/4km\ppt_04
http://services.nacse.org/prism/data/public/normals/4km\ppt_05
http://services.nacse.org/prism/data/public/normals/4km\ppt_06
http://services.nacse.org/prism/data/public/normals/4km\ppt_07
http://services.nacse.org/prism/data/public/normals/4km\ppt_08
http://services.nacse.org/prism/data/public/normals/4km\ppt_09
http://services.nacse.org/prism/data/public/normals/4km\ppt_10
http://services.nacse.org/prism/data/public/normals/4km\ppt_11
http://services.nacse.org/prism/data/public/normals/4km\ppt_12
http://services.nacse.org/prism/data/public/normals/4km\ppt_14
http://services.nacse.org/prism/data/public/normals/4km\tmean_01
http://services.nacse.org/prism/data/public/normals/4km\tmean_02
http://services.nacse.org/prism/data/public/normals

### Unzip files & move to seperate weather variable folders
#### Set variables: 
* data_base_path - Path where unzipped files will go
* directory - Path for downloaded zipped files

In [ ]:
# Set path for unzipped files
from zipfile import ZipFile

data_base_path = r"C:\Users\mmMary\Documents\8990\DATA"  # Path for unzipped files
directory = r"C:\Users\mmMary\Downloads"  # Path for downloaded zipped files

def unzip_climate(element):
    output_data_folder = os.path.join(data_base_path, element)

    for filename in os.listdir(directory):
        if filename.startswith("PRISM_" + element):
            with ZipFile(filename, 'r') as zipObj:
                zipObj.extractall(output_data_folder)

unzip_climate("ppt")

In [134]:
# Set path for unzipped files
from zipfile import ZipFile

data_base_path = r"C:\Users\mmMary\Documents\8990\DATA"  # Path for unzipped files
directory = r"C:\Users\mmMary\Downloads"  # Path for downloaded zipped files

def unzip_climate(element):
    output_data_folder = os.path.join(data_base_path, element)

    try:
        for filename in os.listdir(directory):
            if filename.startswith("PRISM_" + element):
                with ZipFile(filename, 'r') as zipObj:
                    zipObj.extractall(output_data_folder)
    
    except Exception as e:
        print("unable to download:",  e)
        

unzip_climate("ppt")

unable to download: [Errno 2] No such file or directory: 'PRISM_ppt_30yr_normal_4kmM3_02_bil.zip'


### Reduce file name to only include weather variable & timeframe
There are character limitaions for Esri file names that often cause problems. 
Since the downloaded files have lengthy names, this should avoid errors for breakin Esri's naming convention.

climate_variable:
- ppt
- tmax
- tmin
- tmean

In [102]:
# Copy raw data files into a new folder in the temperary data directiory to store the shorter named files
# This preserves to raw data

# import os

def name_reducer(climate_variable):
     
# SET FILE PATH TO TEMPORARY DATA FOLDER
    dir_path = os.path.join(r"C:\Users\mmMary\Documents\8990\T\DATA", climate_variable)
    os.chdir(dir_path)
    
    for f in os.listdir():

    # Split off file extension from filename
        file_name, file_ext =os.path.splitext(f)

    # Split each name component into seperate parts
        prism, weather_type, yr, normal, resolution, month, extension = file_name.split('_')

    # New name combines just the weather type,month, multiple file extensions where applicable & the last file type
        new_name = '{}_{}_{}{}'.format(weather_type, month, extension, file_ext)

    # Rename all files to short names
        os.rename(f, new_name)

    print(weather_type, "renamed files are done")

name_reducer("ppt")
    
#     https://youtu.be/ve2pmm5JqmI



ppt renamed files are done


# Step 2. Reproject raw rasters into Minnesota Coordinate Refence System
#### EPSG 26915 - NAD83 UTM 15N

### The following are my attempts to automate projection into MN UTM CRS

In [152]:
# This isn't working properly !!!! ERROR 999999
import arcpy

climate_variable = "ppt"
# Set the current workspace
arcpy.env.workspace = os.path.join(r"C:\Users\mmMary\Desktop\8990\test\DATA", climate_variable)
spatial_reference_object = arcpy.SpatialReference(26915)

rasters = arcpy.ListRasters()
for raster in rasters:
    print(raster)
    arcpy.management.ProjectRaster(raster, arcpy.env.workspace, spatial_reference_object)
    print(raster + " is now in UTM15 N")
    
# https://pro.arcgis.com/en/pro-app/latest/arcpy/functions/listrasters.htm    
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/project-raster.htm

ppt_01_bil.bil


ExecuteError: ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
Failed to execute (ProjectRaster).


In [ ]:
# Project raw .bil U.S. files into MN CRS ESPG 26915

# This works when running ind layer

# Create project command using spatial reference object

in_raster = r"C:\Users\mmMary\Documents\8990\DATA\Precipitation\PRISM_ppt_30yr_normal_4kmM3_annual_bil.bil"
out_raster = r"C:\Users\mmMary\Documents\8990\T\Project_Rasters\ppt\PRISM_ppt_30yr_normal_4kmM3_annual_bil.bil"
# wkt = 'PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
spatial_reference_object = arcpy.SpatialReference(26915)
arcpy.management.ProjectRaster(in_raster, out_raster, spatial_reference_object)

# Sources:
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/project-raster.htm
# https://pro.arcgis.com/en/pro-app/latest/arcpy/classes/spatialreference.htm
#     https://epsg.io/26915

In [ ]:
# This works when I project one file at a time but doesn't when looping thru folder (see below)

import os
# Set base path for raw rasters
directory = os.fsencode(r"C:\Users\mmMary\Documents\8990\DATA\Precipitation")

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".bil"):
        
# Get file names for all .bil files
#         print(filename)
                            
        base_path = r"C:\Users\mmMary\Documents\8990\T\Project_Rasters\ppt"
        out_raster = os.path.join(base_path, filename)
       
 # Projection system for MN
        spatial_reference_object = arcpy.SpatialReference(26915)
        
# Project all US scale rasters
        arcpy.management.ProjectRaster(filename, out_raster, spatial_reference_object)
        print(output_file)

Step 1: Project the entire U.S. .bil raster into MN CRS ESPG 26915

In [ ]:
# Another attempt to automate the projection. This, sadly, also raises an error (shown at the bottom)
# Try using arcpy.ListRasters

import os
import arcpy
# Set base path for raw rasters
climate_variable = "tmax"

# arcpy.env.workspace = os.path.join(r"C:\Users\mmMary\Documents\8990\T\DATA", climate_variable)
in_raster = os.path.join(r"C:\Users\mmMary\Documents\8990\T\DATA", climate_variable)   
out_raster = os.path.join(r"C:\Users\mmMary\Documents\8990\T\LAYERS\MN_Extent", climate_variable)
arcpy.env.workspace = in_raster

raster_list = arcpy.ListRasters()
for raster in raster_list:
#     print(raster)
    try:  
        # Projection system for MN
        spatial_reference_object = arcpy.SpatialReference(26915)
        
        # Project all US scale rasters
        arcpy.management.ProjectRaster(raster, out_raster, spatial_reference_object)
        print(raster + "in now in UTM 15N")
    except Exception as e:
        print(e)
        
        
"""Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
Failed to execute (ProjectRaster)."""        

In [ ]:
Step 2. Convert metric to standard units - milimeters to inches

Clip raster to Minnesota boundary

In [ ]:
input = 

arcpy.management.Clip("JAN_PRISM_ppt_30yr_normal_4kmM3_annual_bil.bil", "-97.2698643033953 43.4353843588883 -89.3971195198901 49.4045362223117", r"C:\Users\mmMary\Documents\8990\Layers\MN_Precip\Jan", "State", "-9999", "ClippingGeometry", "NO_MAINTAIN_EXTENT")

# Step 3. Clip U.S. MN CRS raster to MN rectangle. 
This is bigger than MN boundary & leaves room to refine the edges after running the Contour tool.

In [ ]:
# Clip projected U.S. raster to MN Rectangle Extent

import arcpy, os

def extent_clip(climate_variable):
    arcpy.env.workspace = os.path.join(r"C:\Users\mmMary\Documents\8990\T\LAYERS\UTM", climate_variable)  
    clip_shp = r"C:\Users\mmMary\Documents\8990\Layers\MN_Boundary\State.shp"

    raster_list = arcpy.ListRasters()
    for raster in raster_list:
        out_raster = os.path.join(r"C:\Users\mmMary\Documents\8990\T\LAYERS\MN_Extent", climate_variable, raster) 
        arcpy.management.Clip(raster, "170463.204 4796993.2567 780967.2014 5491739.865", out_raster, clip_shp, "-9999", "NONE", "MAINTAIN_EXTENT")
        print(raster + " is clipped to MN rectangle")
        
extent_clip("tmin")


# Sources
# https://stackoverflow.com/questions/30043403/create-for-loop-to-process-multiple-rasters-in-a-file-using-arcpy-clip-managemen
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/clip.htm

# Step 4. Convert Rasters from metric to standard units

In [1]:
#import libraries
import arcpy
from arcpy.sa import *
import os

### Convert rain volumes from mm to inches
#### mm / 25.4 = inches

In [ ]:
climate_variable = "precip"
in_raster_path = os.path.join(r"C:\Users\mmMary\Documents\8990\T\LAYERS\MN_Extent", climate_variable)   
arcpy.env.workspace = in_raster_path


rasters = arcpy.ListRasters()
for raster in rasters:
# Directory for converted rasters    
    new_raster = os.path.join(r"C:\Users\mmMary\Documents\8990\T\LAYERS\Standard_Units", climate_variable, raster)  
# Create a raster object
    rasterObject = Raster(raster)
# Convert from mm to inches
    outraster = rasterObject / 25.4
# Check the min & max values to assess quality    
    min_inch = outraster.minimum
    max_inch = outraster.maximum
#Save to new folder
    outraster.save(new_raster)    
    print("Converted to inches: ", raster, "min & max inches: ", min_inch, max_inch)

### Convert temperature values from Celsius to Fahrenheit
#### C° to F°: (Degrees°C x 1.8) + 32 = Degrees°F

In [2]:
# A function to convert temperature from Celsius to Fahrenheit 

def celsius_to_fahren(climate_variable):
    in_raster_path = os.path.join(r"C:\Users\mmMary\Documents\8990\T\LAYERS\MN_Extent", climate_variable)   
    arcpy.env.workspace = in_raster_path
   

    rasters = arcpy.ListRasters()
    for raster in rasters:
            
        # Directory for converted rasters    
        new_raster = os.path.join(r"C:\Users\mmMary\Documents\8990\T\LAYERS\Standard_Units", climate_variable, raster)  
        # Create a raster object
        rasterObject = Raster(raster)
        # Convert from Cesius to Fahrenheit
        outraster = (rasterObject * 1.8) + 32
        # Check the min & max values to assess quality    
        min_temp = outraster.minimum
        max_temp = outraster.maximum
        #Save to new folder
        outraster.save(new_raster)    
        print("Converted to Degrees Fahrenheit: ", raster, "min & max Degrees: ", min_temp, max_temp)



In [ ]:
# Call the function for each temperature variables
celsius_to_fahren("tmin")

# Step 5. Contour Tool to convert to Vector format.

The Contour tool converts rasters into vectors. This eliminates pixellation of the map and converts it into a shapefile to ease file sharing and publication.

#### Contours for temperature:
Monthly normals are split into 2 degree increments while the annual temperature normal is split into increments of 1 inch

In [33]:
def temp_contour(climate_variable):
    arcpy.env.workspace = os.path.join(r"C:\Users\mmMary\Documents\8990\T\LAYERS\Standard_Units", climate_variable)  
    # List to hold names of rasters that were successfully converted
    contour_list = []
    
    raster_list = arcpy.ListRasters()
    for raster in raster_list:
        input_raster = os.path.join(r"C:\Users\mmMary\Documents\8990\T\LAYERS\Standard_Units", climate_variable, raster) 
        output_contour = os.path.join(r"C:\Users\mmMary\Documents\8990\T\LAYERS\Contour", climate_variable, raster + ".shp")
        
        
        arcpy.sa.Contour(
        input_raster, output_contour, 
        2, 0, 1, "CONTOUR_POLYGON", None)
        
        contour_list.append(raster)
        
    ann_raster = arcpy.ListRasters("*ann_bil*")
    for raster in ann_raster:

        arcpy.sa.Contour(
        input_raster, output_contour, 
        1, 0, 1, "CONTOUR_POLYGON", None)
    
        contour_list.append(raster)
      
 # Show list of all converted files to make sure all were successful
    print(contour_list, " is now contoured")

# Call the function for each temperature variable    
temp_contour("tmax")    

['tmax_01_bil', 'tmax_02_bil', 'tmax_03_bil', 'tmax_04_bil', 'tmax_05_bil', 'tmax_06_bil', 'tmax_07_bil', 'tmax_08_bil', 'tmax_09_bil', 'tmax_10_bil', 'tmax_11_bil', 'tmax_12_bil', 'tmax_ann_bil', 'tmax_ann_bil']  is now contoured


#### Contours for precipitation:
Monthly normals are split into 0.25 inch increments while the annual precip normal is split into increments of 2 inches

In [14]:
import arcpy, os
from arcpy.sa import *

def precip_contour(climate_variable):
    arcpy.env.workspace = os.path.join(r"C:\Users\mmMary\Documents\8990\T\LAYERS\Standard_Units", climate_variable)  
    # List to hold names of rasters that were successfully converted
    contour_list = []
    
# Convert all of the monthly rasters using the 0.25 increment    
    raster_list = arcpy.ListRasters()
    for raster in raster_list:
        input_raster = os.path.join(r"C:\Users\mmMary\Documents\8990\T\LAYERS\Standard_Units", climate_variable, raster) 
        output_contour = os.path.join(r"C:\Users\mmMary\Documents\8990\T\LAYERS\Contour", climate_variable, raster + ".shp")
        
        arcpy.sa.Contour(
        input_raster, output_contour, 
        0.25, 0, 1, "CONTOUR_POLYGON", None)
        
        contour_list.append(raster)

# Handle the annual raster with 2 inch increment
    ann_raster = arcpy.ListRasters("*ann_bil*")
    for raster in ann_raster:
        
        arcpy.sa.Contour(
        input_raster, output_contour, 
        2, 15, 1, "CONTOUR_POLYGON", None)
        
        contour_list.append(raster)

# Print out confirmation list to ensure all files were successfully converted
    print(output_contour + " is now contoured")

# Call the function for precipitation
precip_contour("precip")    
       

C:\Users\mmMary\Documents\8990\T\LAYERS\Contour\precip\ppt_ann_bil.shp is now contoured


#### Sources:
https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/contour.htm

# Step 6. Clip to Minnesota State Boundary
This clips it to the exact MN boundary while preserving clean edges and data.

In [38]:
# Clip projected U.S. raster to MN Rectangle Extent

def MN_clip(climate_variable):
    arcpy.env.workspace = os.path.join(r"C:\Users\mmMary\Documents\8990\T\LAYERS\Contour", climate_variable)  
    clip_feature = r"C:\Users\mmMary\Documents\8990\Layers\MN_Boundary\State.shp"

    feature_classes = arcpy.ListFeatureClasses()
    for fc in feature_classes:
        in_feature = os.path.join(arcpy.env.workspace, fc)
        out_feature = os.path.join(r"C:\Users\mmMary\Documents\8990\Layers\MN_Vector_Varibles", climate_variable, fc) 
        
        arcpy.analysis.Clip(in_feature, clip_feature, out_feature, None)
        print(fc + " is clipped to MN boundary")
        
                
MN_clip("precip")

ppt_01_bil.shp is clipped to MN boundary
ppt_02_bil.shp is clipped to MN boundary
ppt_03_bil.shp is clipped to MN boundary
ppt_04_bil.shp is clipped to MN boundary
ppt_05_bil.shp is clipped to MN boundary
ppt_06_bil.shp is clipped to MN boundary
ppt_07_bil.shp is clipped to MN boundary
ppt_08_bil.shp is clipped to MN boundary
ppt_09_bil.shp is clipped to MN boundary
ppt_10_bil.shp is clipped to MN boundary
ppt_11_bil.shp is clipped to MN boundary
ppt_12_bil.shp is clipped to MN boundary
ppt_ann_bil.shp is clipped to MN boundary


# Step 7. Apply Symbology
Set the symbology based on precipitation, temperature, monthly & annual values.

Create a symbology layer for the monthly and annual precipitaion and temperature.
Save each as a Layer File

In [ ]:
# This applies symbology from another layer that must be stored 

# Apply symbology from layer

arcpy.env.workspace = r"C:\Users\mmMary\Documents\8990\T\LAYERS\symbology\precip"

ppt_ann_style = r"C:\Users\mmMary\Documents\8990\Layers\Symbology_Layers\ppt_ann_style.shp"
ppt_monthly = r"C:\Users\mmMary\Documents\8990\T\LAYERS\Layer_Styles\ppt_monthly.shp"

featureclasses = arcpy.ListFeatureClasses()

for fc in featureclasses:
    if fc.endswith("_ann_bil.shp"):
        arcpy.management.ApplySymbologyFromLayer(fc, "ppt_ann_style", 
                                         "VALUE_FIELD ContourMin ContourMin", "MAINTAIN")
    else:
#         print(fc)
        arcpy.management.ApplySymbologyFromLayer(fc, "ppt_monthly", 
                                         "VALUE_FIELD ContourMin ContourMin", "MAINTAIN")
    
# https://pro.arcgis.com/en/pro-app/latest/arcpy/mapping/uniquevaluerenderer-class.htm

# Step 8. Export to image


This step requires layout design to be done in ArcGIS Pro & manual manipulation of the layer files. 
Add the legend, title & other desired attributes then run this cell.

#### !! Run the cell once for each layer - turning off the top layer beforehand so each iteration is captured in the final image. !!

# This works but requires a lot of manual input between each layer. 
# It does however print the final layout complete with legend & titles

In [10]:
import os
aprx = arcpy.mp.ArcGISProject(r"C:\Users\mmMary\Documents\8990\Map\Weather_Map\Weather_Map.aprx")

# Update these parameters each instance for proper naming convention.
# !! Failing to do will overwrite the last file !!
climate_variable = "ppt"
time = "01"

# Output_File = os.path.join(r"C:\Users\mmMary\Documents\8990\Static_Maps", climate_variable, climate_variable + "_" + time) 

Output_File = os.path.join(r"C:\Users\mmMary\Desktop\8990", climate_variable, climate_variable + "_" + time + ".jpg") 
m = aprx.listMaps("Print")[0]
lyt = aprx.listLayouts("Layout")[0]
lyt.exportToJPEG(Output_File) #, width=1000, height=1000, world_file=True) 

'C:\\Users\\mmMary\\Desktop\\8990\\ppt\\ppt_01.jpg'

# The below cell is still a work in progress but it seems like I'm close. 

In [20]:
# THIS is getting close but still not quite correct. Still working on it
import arcpy, os

aprx = arcpy.mp.ArcGISProject('current')
mv = aprx.activeView

# Loop thru all layers in mapframe
layers = m.listLayers()

for l in layers:
    # Get layer as string for filename
    l_name = str(l)
    
    # Export the map view
    Output_File = os.path.join(r"C:\Users\mmMary\Documents\8990\Static_Maps\testing", l_name) 
    mv.exportToJPEG(Output_File, width=1000, height=1000, world_file=True)
    
    print(Output_File + " is now an image")
#     print(l)

Counties
State
ppt_monthly
ppt_ann_style
ppt_12_bil_Layer
ppt_11_bil_Layer
ppt_10_bil_Layer
ppt_09_bil_Layer
ppt_08_bil_Layer
ppt_07_bil_Layer
ppt_06_bil_Layer
ppt_05_bil_Layer
ppt_04_bil_Layer
ppt_03_bil_Layer
ppt_02_bil_Layer
ppt_01_bil_Layer1
ppt_12_bil_Layer
ppt_ann_bil


#### Sources:
https://pro.arcgis.com/en/pro-app/latest/tool-reference/analysis/clip.htm ,/n https://pro.arcgis.com/en/pro-app/latest/arcpy/functions/listfeatureclasses.htm